<a href="https://colab.research.google.com/github/bhaghighi/Deep-Learning-101/blob/master/Segmentation/UNet_Segmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This notebook describe the neccesary steps for simple UNet segmentation
<br>
The contents are got from this source: "https://github.com/usuyama/pytorch-unet"


**Making Synthetic images for segmentation** 